# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [5]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from config import g_key




### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [6]:
pd_readcsv_df = pd.read_csv("../output_data/cities.csv")
pd_readcsv_df.head()


,Unnamed: 0,City,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Dolores Hidalgo Cuna de la Independencia Nacional,21.16,-100.93,60.80,44,5,2.24,MX,1604979105
1,1,Mar del Plata,-38.00,-57.56,62.01,87,0,3.36,AR,1604979113
2,2,Ambilobe,-13.20,49.05,80.58,53,52,6.49,MG,1604979115
3,3,Bambous Virieux,-20.34,57.76,77.00,73,75,10.29,MU,1604979116
4,4,Albany,42.60,-73.97,51.01,87,20,1.57,US,1604978958


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [7]:
gmaps.configure(api_key=g_key)


In [8]:
locations = pd_readcsv_df[['Latitude', 'Longitude']]
humidity = pd_readcsv_df['Humidity']
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, max_intensity = 200, point_radius = 3, dissipating = False)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.


  * A max temperature lower than 80 degrees but higher than 70.

  * Wind speed less than 10 mph.

  * Zero cloudiness.

  * Drop any rows that don't contain all three conditions. You want to be sure the weather is ideal.

In [9]:
weather_crit_df = pd_readcsv_df.loc[(pd_readcsv_df['Max Temp'] < 80) & (pd_readcsv_df['Max Temp'] > 75) &
                                    (pd_readcsv_df['Wind Speed'] < 10) &
                                    (pd_readcsv_df['Cloudiness'] == 0)]
weather_crit_df.dropna()
weather_crit_df


,Unnamed: 0,City,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
141,141,Nizwá,22.93,57.53,76.57,43,0,1.95,OM,1604979272
179,179,Tharād,24.40,71.63,79.52,22,0,5.97,IN,1604979315
264,264,Alta Floresta,-9.88,-56.09,77.00,94,0,1.12,BR,1604979413
265,265,Rajanpur,29.10,70.33,77.02,18,0,8.01,PK,1604979414
280,280,Yārāda,17.65,83.27,78.80,50,0,8.05,IN,1604979431
281,281,Atar,20.52,-13.05,79.61,16,0,6.93,MR,1604979432
302,302,Srīvardhan,18.03,73.02,77.92,42,0,4.85,IN,1604979456
315,315,Belmonte,-15.86,-38.88,75.20,78,0,8.05,BR,1604979471
356,356,Adrar,20.50,-10.07,76.17,19,0,7.20,MR,1604979518
423,423,Ormara,25.21,64.64,77.68,33,0,5.23,PK,1604979594


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [16]:
hotel_df = weather_crit_df

In [17]:
hotel_df["Hotel Name"] = ""
hotel_df_clean = hotel_df[["City", "Latitude", "Longitude", "Country", "Hotel Name"]]
hotel_df_clean




C:\ProgramData\Anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,City,Latitude,Longitude,Country,Hotel Name
141,Nizwá,22.93,57.53,OM,
179,Tharād,24.40,71.63,IN,
264,Alta Floresta,-9.88,-56.09,BR,
265,Rajanpur,29.10,70.33,PK,
280,Yārāda,17.65,83.27,IN,
281,Atar,20.52,-13.05,MR,
302,Srīvardhan,18.03,73.02,IN,
315,Belmonte,-15.86,-38.88,BR,
356,Adrar,20.50,-10.07,MR,
423,Ormara,25.21,64.64,PK,


In [18]:
params = {'radius' : 5000,
          'types' : "hotel",
          'key' : g_key}

for index, row in hotel_df_clean.iterrows():
    latitude = row['Latitude']
    longitude = row['Longitude']
    params["location"] = f"{latitude},{longitude}"
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    address = requests.get(base_url, params = params).json()
    try:
        hotel_df_clean.loc[index, "Hotel Name"] = address["results"][0]["name"]
    except: 
        print("Hotel not found")
                   

                   
    

C:\ProgramData\Anaconda3\envs\PythonData\lib\site-packages\pandas\core\indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


Hotel not found


In [19]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df_clean.iterrows()]
locations = hotel_df_clean[["Latitude", "Longitude"]]

In [20]:
# Add marker layer ontop of heat map
fig2 = gmaps.figure()
marker_layer = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig2.add_layer(marker_layer)


# Display figure
fig2

Figure(layout=FigureLayout(height='420px'))